# **Yelp API Core**

_John Andrew Dixon_

---

##### **Imports**

In [2]:
import json
import pandas as pd

---

## **Load API Key**

In [4]:
with open('/Users/johna/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])